In [28]:
import random

In [145]:
class Island():
    
    def __init__(self, name):
        self.name = name
        self.matrix = [['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X'],
                       ['X', '-', '-', '-', '-', '-', '-', 'X'],
                       ['X', '-', '-', '-', '-', '-', '-', 'X'],
                       ['X', '-', '-', '-', '-', '-', '-', 'X'],
                       ['X', '-', '-', '-', '-', '-', '-', 'X'],
                       ['X', '-', '-', '-', '-', '-', '-', 'X'],
                       ['X', '-', '-', '-', '-', '-', '-', 'X'],
                       ['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X']]
        
    def place(self, thing, spot):
        self.thing = thing
        self.spot = spot
        
        self.matrix[self.spot[0]][self.spot[1]] = self.thing
        
    def percept(self, agent):
        self.agent = agent
        self.percepts = []
        
        here = (self.agent.location[0], self.agent.location[1])
        up = (self.agent.location[0] - 1, self.agent.location[1])
        upright = (self.agent.location[0] - 1, self.agent.location[1] + 1)
        right = (self.agent.location[0], self.agent.location[1] + 1)
        downright = (self.agent.location[0] + 1, self.agent.location[1] + 1)
        down = (self.agent.location[0] + 1, self.agent.location[1])
        downleft = (self.agent.location[0] + 1, self.agent.location[1] - 1)
        left = (self.agent.location[0], self.agent.location[1] - 1)
        upleft = (self.agent.location[0] - 1, self.agent.location[1] - 1)
        
        self.percepts.append((self.matrix[here[0]][here[1]], here))
        self.percepts.append((self.matrix[up[0]][up[1]], up))
        self.percepts.append((self.matrix[upright[0]][upright[1]], upright))
        self.percepts.append((self.matrix[right[0]][right[1]], right))
        self.percepts.append((self.matrix[downright[0]][downright[1]], downright))
        self.percepts.append((self.matrix[down[0]][down[1]], down))
        self.percepts.append((self.matrix[downleft[0]][downleft[1]], downleft))
        self.percepts.append((self.matrix[left[0]][left[1]], left))
        self.percepts.append((self.matrix[upleft[0]][upleft[1]], upleft))
        
    def run(self, agent, iterations):
        self.agent = agent
        self.iterations = iterations
        
        print("Starting...")
        print("Agent's location: " + str(self.agent.location))
        print("Agent's tools: " + str(self.agent.inventory))
        print("Agent's performance: " + str(self.agent.performance))
        print("Initial state of the environment:")
        print('\n'.join([''.join(['{:4}'.format(item) for item in row]) 
                for row in self.matrix]))
        
        for i in range(1, self.iterations):
            print("<STEP" + str(i) + ">")
            
            self.percept(self.agent)
            print("PERCEPT")
            print(self.percept)
            
            action = self.agent.program(self.percepts)
            
            if action == True:
                self.matrix[self.agent.location[0]][self.agent.location[1]] = "-"
                
            print("New agent's performance: " + str(self.agent.performance))
            print("NEW ENVIRONMENT STATE")
            print("Agent's location: " + str(self.agent.location))
            print("Agent's tools: " + str(self.agent.inventory))
            print('\n'.join([''.join(['{:4}'.format(item) for item in row]) 
                  for row in self.matrix]))
        
        print("Run finished. Final agent's performance was: " + str(self.agent.performance))
        

In [142]:
class Agent():
    
    def __init__(self, name):
        self.name = name
        self.location = (1, 1)
        self.inventory = {'reusable_tools': 0, 'disposable_tools': 0}
        self.performance = 50
        
    def program(self, percept):
        self.percept = percept
        print(self.percept)
        
        current = self.percept[0][0]
        if current == 'h':
            self.gdispos()
            return True
        
        elif current == 'H':
            self.greuse()
            return True
        
        elif current == 't':
            if self.inventory['disposable_tools'] > 0:
                self.gtreas2()
                return True
            else:
                print("Due to no disposable tools, did not try to grab a treasure type 2 at current location " + str(self.location))
                self.noop()
                return False
            
        elif current == 'T':
            if self.inventory['reusable_tools'] > 0 or self.inventory['disposable_tools'] > 0:
                self.gtreas1()
                return True
            else:
                print("Due to no tools, did not try to grab a treasure type 1 at current location " + str(self.location))
                self.noop()
                return False
        
        for p in self.percept:
            if p[0] == 'h':
                target = p[1]
                self.directed_move(target)
                break
            
        else:
            for p in self.percept:
                if p[0] == 'H':
                    target = p[1]
                    self.directed_move(target)
                    break
            else:
                for p in self.percept:
                    if p[0] == 't':
                        target = p[1]
                        self.directed_move(target)
                        break
                else:
                    for p in self.percept:
                        if p[0] == 'T':
                            target = p[1]
                            self.directed_move(target)
                            break
                    else:
                        self.random_move()
                
    def random_move(self):
        del self.percept[0]
        temp = []
        
        for p in self.percept:
            if p[0] != 'X':
                if p[1][0] == self.location[0] or p[1][1] == self.location[1]:
                    temp.append(p) 
       
        moves = []
        
        for p in range(0, len(temp)):
            moves.append(temp[p][1])
            
        self.location = random.choice(moves)
        print("Moved randomly to " + str(self.location))
    
    def directed_move(self, target):
        self.target = target
        
        if self.target[0] != self.location[0] and self.target[1] != self.target[1]:
            if self.target[0] - self.location[0] > 0:
                self.down()
            else:
                self.up()
                
        elif self.target[0] == self.location[0] - 1:
            self.up()
            
        elif self.target[1] == self.location[1] + 1:
            self.right()
            
        elif self.target[0] == self.location[0] + 1:
            self.down()
            
        elif self.target[1] == self.location[1] - 1:
            self.left()
            
    def up(self):
        self.location = (self.location[0] - 1, self.location[1])
        print("Moved up to " + str(self.location))
        self.performance -= 1
    
    def right(self):
        self.location = (self.location[0], self.location[1] + 1)
        print("Moved right to " + str(self.location))
        self.performance -= 1
        
    def left(self):
        self.location = (self.location[0], self.location[1] - 1)
        print("Moved left to " + str(self.location))
        self.performance -= 1
        
    def down(self):
        self.location = (self.location[0] + 1, self.location[1])
        print("Moved down to " + str(self.location))
        self.performance -= 1
        
    def greuse(self):
        self.inventory['reusable_tools'] += 1
        print("Grabbed a reusable tool at current location "+ str(self.location))
        self.performance -= 2

    def gdispos(self):
        self.inventory['disposable_tools'] += 1
        print("Grabbed a disposable tool at current location "+ str(self.location))
        self.performance -= 2
    
    def gtreas2(self):
        self.inventory['disposable_tools'] -= 1
        print("Using a disposable tool, grabbed a treasure type 2 at current location " + str(self.location))
        self.performance += 40
        
    def noop(self):
        print("Stayed at location " + str(self.location))
        
    def gtreas1(self):
        if self.inventory['reusable_tools'] > 0:
            print("Using a reusable tool, grabbed a treasure type 1 at current location " + str(self.location))
            self.performance += 20
        else:
            print("Using a disposable tool, grabbed a treasure type 1 at current location " + str(self.location))
            self.performance += 20


In [151]:
test_island = Island("test_island")
test_hunter = Agent("test_hunter")

test_island.place("h", (1, 2))
test_island.place("h", (2, 3))
test_island.place("H", (2, 2))
test_island.place("t", (3, 2))
test_island.place("t", (4, 2))
test_island.place("T", (5, 2))
test_island.place("T", (6, 6))
test_island.run(test_hunter, 50)


Starting...
Agent's location: (1, 1)
Agent's tools: {'reusable_tools': 0, 'disposable_tools': 0}
Agent's performance: 50
Initial state of the environment:
X   X   X   X   X   X   X   X   
X   -   h   -   -   -   -   X   
X   -   H   h   -   -   -   X   
X   -   t   -   -   -   -   X   
X   -   t   -   -   -   -   X   
X   -   T   -   -   -   -   X   
X   -   -   -   -   -   T   X   
X   X   X   X   X   X   X   X   
<STEP1>
PERCEPT
<bound method Island.percept of <__main__.Island object at 0x7f3d2776e668>>
[('-', (1, 1)), ('X', (0, 1)), ('X', (0, 2)), ('h', (1, 2)), ('H', (2, 2)), ('-', (2, 1)), ('X', (2, 0)), ('X', (1, 0)), ('X', (0, 0))]
Moved right to (1, 2)
New agent's performance: 49
NEW ENVIRONMENT STATE
Agent's location: (1, 2)
Agent's tools: {'reusable_tools': 0, 'disposable_tools': 0}
X   X   X   X   X   X   X   X   
X   -   h   -   -   -   -   X   
X   -   H   h   -   -   -   X   
X   -   t   -   -   -   -   X   
X   -   t   -   -   -   -   X   
X   -   T   -   -   -   -   X  